In [2]:
#Bibliotecas para poder trabajar con Spark
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
#Configuración de Spark con Python
!pip install -q findspark
!pip install pyspark

#Estableciendo variable de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

#Buscando e inicializando la instalación de Spark
import findspark
findspark.init()
findspark.find()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [591 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,012 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,185 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,389 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://ppa.launchpadco

'/content/spark-3.5.0-bin-hadoop3'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import DoubleType
from pyspark.sql import Row
from pyspark.sql.functions import col

In [10]:
# Iniciar la sesión de Spark
spark = SparkSession.builder.appName("MovieRecommendations").getOrCreate()

In [31]:
# Cargar datos desde un archivo CSV
df = spark.read.csv("/content/drive/My Drive/Colab Notebooks/netflix_titles.csv", header=True, inferSchema=True)
df.show()

+--------+-------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+--------+--------+--------------------+--------------------+
| show_id|   type|               title|            director|                cast|             country|       date_added|release_year|  rating|duration|           listed_in|         description|
+--------+-------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+--------+--------+--------------------+--------------------+
|81145628|  Movie|Norm of the North...|Richard Finn, Tim...|Alan Marriott, An...|United States, In...|September 9, 2019|        2019|   TV-PG|  90 min|Children & Family...|Before planning a...|
|80117401|  Movie|Jandino: Whatever...|                NULL|    Jandino Asporaat|      United Kingdom|September 9, 2016|        2016|   TV-MA|  94 min|     Stand-Up Comedy|"Jandino Asporaat...|
|70234439|TV Show|  Transforme

In [33]:
# Select the columns you need
df = df.select("title", "description")

# Filter out rows with null descriptions
df = df.filter(col("description").isNotNull())
df.show()

+--------------------+--------------------+
|               title|         description|
+--------------------+--------------------+
|Norm of the North...|Before planning a...|
|Jandino: Whatever...|"Jandino Asporaat...|
|  Transformers Prime|With the help of ...|
|Transformers: Rob...|When a prison shi...|
|        #realityhigh|When nerdy high s...|
|             Apaches|A young journalis...|
|            Automata|In a dystopian fu...|
|Fabrizio Copano: ...|Fabrizio Copano t...|
|        Fire Chasers|As California's 2...|
|         Good People|A struggling coup...|
|Joaquín Reyes: Un...|Comedian and cele...|
|Kidnapping Mr. He...|"When beer magnat...|
|Krish Trish and B...|A team of minstre...|
|Krish Trish and B...|An artisan is che...|
|Krish Trish and B...|A cat, monkey and...|
|Krish Trish and B...|In three comic-st...|
|Krish Trish and B...|A cat, monkey and...|
|Krish Trish and B...|Animal minstrels ...|
|Krish Trish and B...|The consequences ...|
|                Love|A man in a

In [34]:
# Tokenize the descriptions
tokenizer = Tokenizer(inputCol="description", outputCol="words")
df = tokenizer.transform(df)
df.show()

+--------------------+--------------------+--------------------+
|               title|         description|               words|
+--------------------+--------------------+--------------------+
|Norm of the North...|Before planning a...|[before, planning...|
|Jandino: Whatever...|"Jandino Asporaat...|["jandino, aspora...|
|  Transformers Prime|With the help of ...|[with, the, help,...|
|Transformers: Rob...|When a prison shi...|[when, a, prison,...|
|        #realityhigh|When nerdy high s...|[when, nerdy, hig...|
|             Apaches|A young journalis...|[a, young, journa...|
|            Automata|In a dystopian fu...|[in, a, dystopian...|
|Fabrizio Copano: ...|Fabrizio Copano t...|[fabrizio, copano...|
|        Fire Chasers|As California's 2...|[as, california's...|
|         Good People|A struggling coup...|[a, struggling, c...|
|Joaquín Reyes: Un...|Comedian and cele...|[comedian, and, c...|
|Kidnapping Mr. He...|"When beer magnat...|["when, beer, mag...|
|Krish Trish and B...|A t

In [35]:
# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words") # Create a StopWordsRemover object to remove common stop words from the 'words' column
df = remover.transform(df) # Apply the StopWordsRemover to the DataFrame 'df' and create a new column 'filtered_words'
df.show()

+--------------------+--------------------+--------------------+--------------------+
|               title|         description|               words|      filtered_words|
+--------------------+--------------------+--------------------+--------------------+
|Norm of the North...|Before planning a...|[before, planning...|[planning, awesom...|
|Jandino: Whatever...|"Jandino Asporaat...|["jandino, aspora...|["jandino, aspora...|
|  Transformers Prime|With the help of ...|[with, the, help,...|[help, three, hum...|
|Transformers: Rob...|When a prison shi...|[when, a, prison,...|[prison, ship, cr...|
|        #realityhigh|When nerdy high s...|[when, nerdy, hig...|[nerdy, high, sch...|
|             Apaches|A young journalis...|[a, young, journa...|[young, journalis...|
|            Automata|In a dystopian fu...|[in, a, dystopian...|[dystopian, futur...|
|Fabrizio Copano: ...|Fabrizio Copano t...|[fabrizio, copano...|[fabrizio, copano...|
|        Fire Chasers|As California's 2...|[as, califo

In [36]:
# Create a CountVectorizer to convert tokenized words to a numerical vector
# Specify a maximum vocabulary size of 10,000 words and a minimum document frequency of 2.0
cv = CountVectorizer(inputCol="filtered_words", outputCol="raw_features", vocabSize=10000, minDF=2.0)

# Create an IDF (Inverse Document Frequency) model to transform the raw features
# into weighted features to emphasize important words
idf = IDF(inputCol="raw_features", outputCol="features")

# Create a pipeline to chain the CountVectorizer and IDF stages
pipeline = Pipeline(stages=[cv, idf])

In [37]:
# Fit the pipeline model to the DataFrame
pipeline_model = pipeline.fit(df)

# Transform the DataFrame using the fitted pipeline model
df = pipeline_model.transform(df)

In [39]:
df.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|         description|               words|      filtered_words|        raw_features|            features|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Norm of the North...|Before planning a...|[before, planning...|[planning, awesom...|(9572,[10,20,30,7...|(9572,[10,20,30,7...|
|Jandino: Whatever...|"Jandino Asporaat...|["jandino, aspora...|["jandino, aspora...|(9572,[45,208,229...|(9572,[45,208,229...|
|  Transformers Prime|With the help of ...|[with, the, help,...|[help, three, hum...|(9572,[17,18,149,...|(9572,[17,18,149,...|
|Transformers: Rob...|When a prison shi...|[when, a, prison,...|[prison, ship, cr...|(9572,[2,104,198,...|(9572,[2,104,198,...|
|        #realityhigh|When nerdy high s...|[when, nerdy, hig...|[nerdy, high, sch...|(9572,[29,114,407..

In [40]:
# Calculate cosine similarity

# Create a StringIndexer to assign unique numerical indices to movie titles
indexer = StringIndexer(inputCol="title", outputCol="title_index")

# Fit the StringIndexer to the DataFrame and transform it, adding a new "title_index" column
indexed_df = indexer.fit(df).transform(df)

In [41]:
indexed_df.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|               title|         description|               words|      filtered_words|        raw_features|            features|title_index|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|Norm of the North...|Before planning a...|[before, planning...|[planning, awesom...|(9572,[10,20,30,7...|(9572,[10,20,30,7...|     3633.0|
|Jandino: Whatever...|"Jandino Asporaat...|["jandino, aspora...|["jandino, aspora...|(9572,[45,208,229...|(9572,[45,208,229...|     2490.0|
|  Transformers Prime|With the help of ...|[with, the, help,...|[help, three, hum...|(9572,[17,18,149,...|(9572,[17,18,149,...|     5698.0|
|Transformers: Rob...|When a prison shi...|[when, a, prison,...|[prison, ship, cr...|(9572,[2,104,198,...|(9572,[2,104,198,...|     5702.0|
|        #realityhig

In [42]:
# Create a VectorAssembler to combine columns into a single feature vector
assembler = VectorAssembler(inputCols=["title_index", "features"], outputCol="all_features")

# Apply the VectorAssembler to the DataFrame 'indexed_df' and create a new column 'all_features'
vectorized_df = assembler.transform(indexed_df)

In [43]:
# Create a new DataFrame that includes only the "title" and "all_features" columns
similarity = vectorized_df.select("title", "all_features")

In [46]:
similarity.show(5)

+--------------------+--------------------+
|               title|        all_features|
+--------------------+--------------------+
|Norm of the North...|(9573,[0,11,21,31...|
|Jandino: Whatever...|(9573,[0,46,209,2...|
|  Transformers Prime|(9573,[0,18,19,15...|
|Transformers: Rob...|(9573,[0,3,105,19...|
|        #realityhigh|(9573,[0,30,115,4...|
+--------------------+--------------------+
only showing top 5 rows



In [44]:
# Function to recommend similar movies
def recommend(movie_title, num_recommendations=5):
    # Filter the 'similarity' DataFrame to find the movie data for the given 'movie_title'
    movie_data = similarity.filter(similarity["title"] == movie_title)

    # Check if the movie with the given title exists
    if movie_data.count() == 0:
        print(f"Movie '{movie_title}' not found.")
        return

    # Extract the feature vector of the given movie
    movie_features = movie_data.select("all_features").first()[0]

    # Filter out the given movie from the 'similarity' DataFrame
    similarity_df = similarity.filter(similarity["title"] != movie_title)

    # Calculate the distance (cosine similarity) between the given movie and all other movies
    similarity_df = similarity_df.rdd.map(lambda row: Row(title=row.title, distance=float(Vectors.squared_distance(row.all_features, movie_features))))

    # Convert the RDD back to a DataFrame
    similarity_df = spark.createDataFrame(similarity_df)

    # Sort the movies by similarity distance in ascending order
    similarity_df = similarity_df.sort("distance")

    # Select the top 'num_recommendations' similar movies
    recommended_movies = similarity_df.limit(num_recommendations)

    # Display the recommended movies
    recommended_movies.show()

In [45]:
# EJEMPLO DE USO
recommend("Accident")

+--------------------+-----------------+
|               title|         distance|
+--------------------+-----------------+
|    Act of Vengeance|841.3747063173325|
|          About Time|841.6759018595696|
|Adam Devine: Best...|863.0761671116576|
|           Abdo Mota|871.2622967678093|
|Abominable Christmas|896.2754895487424|
+--------------------+-----------------+

